In [ ]:
import pandas as pd
import numpy as np
import sys
import os
from os import listdir


pd.set_option('display.max_columns', 500)



def find_and_append_module_path():
    current_dir = os.getcwd()
    substring_to_find = 'statsfaction'
    index = current_dir.rfind(substring_to_find)
    
    if index != -1:
        # Extract the directory path up to and including the last "mypath" occurrence
        new_dir = current_dir[:index + (len(substring_to_find))]

        # Change the current working directory to the new directory
        os.chdir(new_dir)
        sys.path.append(new_dir)
        # Verify the new current directory
        print("New current directory:", os.getcwd())
    else:
        print("No 'mypath' found in the current directory")
find_and_append_module_path()


In [ ]:
os.getcwd()

In [ ]:
import requests
from bs4 import BeautifulSoup

# Send an HTTP GET request to the website
url = "https://fminside.net/players"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the main container element containing player data
    player_container = soup.find('div', class_='players')
    
    # Find all player elements within the container
    player_ul_elements = player_container.find_all('ul', class_='player')
    
    # Initialize a list to store player data
    players_data = []
    
    # Loop through each player <ul> element and extract the desired information
    for player_ul in player_ul_elements:
        # Extract player data here
        # Modify the code to extract name, ID, age, rating, and potential
        
        name = player_ul.find('span', class_='name').a.get('title')
        player_id = player_ul.find('span', class_='name').a.get('href')
        age = int(player_ul.find('li', class_='age').text)
        rating = int(player_ul.find('li', class_='rating').span.text)
        potential = int(player_ul.find('li', class_='potential').span.text)
        
        player_info = {
            'Name': name,
            'Player ID': player_id,
            'Age': age,
            'Rating': rating,
            'Potential': potential
        }
        
        # Append the player's data to the list
        players_data.append(player_info)
    
    # Print or process the extracted player data
    for player_info in players_data:
        print(player_info)
else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)


# Load more

In [6]:
import asyncio
from pyppeteer import launch

async def scrape_data():
    # Launch a headless Chromium browser
    browser = await launch(headless=True)

    # Create a new page/tab
    page = await browser.newPage()

    # Navigate to the website
    await page.goto('https://fminside.net/players')

    # Initialize a list to store player data
    players_data = []

    while True:
        # Wait for the player data to load
        await page.waitForSelector('ul.player')

        # Extract player data from the current page using JavaScript evaluation
        player_data = await page.evaluate('''() => {
            const playerElements = Array.from(document.querySelectorAll('ul.player'));
            const data = [];

            for (const playerElement of playerElements) {
                const name = playerElement.querySelector('span.name a').getAttribute('title');
                const playerID = playerElement.querySelector('span.name a').getAttribute('href');
                const age = parseInt(playerElement.querySelector('li.age').textContent, 10);
                const rating = parseInt(playerElement.querySelector('li.rating').textContent, 10);
                const potential = parseInt(playerElement.querySelector('li.potential').textContent, 10);

                data.push({
                    'Name': name,
                    'Player ID': playerID,
                    'Age': age,
                    'Rating': rating,
                    'Potential': potential
                });
            }

            return data;
        }''')

        # Add the player data to the array
        players_data.extend(player_data)

        # Check if there's a "Load more players" button
        load_more_button = await page.querySelector('a.loadmore')
        if not load_more_button:
            break  # No more players to load

        # Click the "Load more players" button
        await load_more_button.click()

        # Wait briefly to allow new content to load
        await page.waitForTimeout(2000)  # Adjust the wait time as needed

    # Close the browser
    await browser.close()

    # Print or process the scraped player data
    for player_info in players_data:
        print(player_info)

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(scrape_data())


RuntimeError: This event loop is already running

In [ ]:
load_more_link

In [ ]:
players_data